# Lab 5 - Bayesian Inference

In the 5th lab of the course, we will study Bayesian Inference in practice.

We will explore the use of Bayesian inference thought a Decision Making example.

# Assignment: Bayesian Decision-Making

## N Meteorologist Problem

In this assignment, we explore the challenge of dealing with predictions from N different meteorologist, each forecasting whether it will rain the next day. Specifically, each meteorology predicts the propability
of rain for the following day.

To be more precise, we can interpret each meteorology as a model that predicts whether it will rain or not. Mathematically, this is expressed as $P(y | M_i)$ for each $i \in M$, where $M$ is the set of possible meteorologist. Here, the random variable $y$ indicates whether it will rain or not (where $y = 1$ means rain and $y = 0$ means no rain). The output of each model, $P(y = 1 | M_i)$, represents the probability of rain for the next day.

For the purposes of this exercise, we assume that we have $N = 3$ different meteorologist for simplicity.  
The set $M$ is defined as $M = \{M1, M2, M3\}$.

In [1]:
import numpy as np

# probability of raining for each meteorologist 10% rain for M1, 50% for M2, 70% for M3
probability_rain = np.array([0.1, 0.5, 0.7]) # p( y=1 | M )

probability_model = np.array([1-probability_rain, probability_rain]) # p(y|M)

In [2]:
probability_model

array([[0.9, 0.5, 0.3],
       [0.1, 0.5, 0.7]])

In [3]:
probability_model[0,0] # P(y=0|M) Model 1 no rain

0.9

In [4]:
probability_model[1,0] # P(y=1|M) Model 1 rain

0.1

# Step 1.1 Marginal Probability

### Marginal Model
In the Bayesian interpretation, we assume that we have **a belief** $P_\beta(M_i)$ about which model $M_i$ is true.

One way to estimate the probability of rain is to use the Bayesian Marginal Model:

$$p_{\text{marginal}}(y) =  \sum_{i \in M} p(y \mid M_i) \cdot P_\beta(M_i)$$

The marginal prediction is the average the probability from each meteorological model, weighted by our belief.  
In other words, it is weighted by how much we trust each model.

### Decision-Making Problem
Now imagine that you have to decide whether it is going to rain and inform some friends.  
Let's denote your decision with $a$: if it's going to rain, $a = 1$, and if it's not, $a = 0$.

Given the $p_{\text{marginal}}(y)$, we can select the final action $a$ that maximizes the marginal model:

$$a = \arg \max_{y} \; p_{\text{marginal}}(y)$$

**Complete the following steps**:  
Let's assume that, on the first day, we have no information about which meteorological model is the best.  
In other words, our prior belief is $P_\beta(M_i) = \text{Prior}(M_i) = 1/N$ for every $i \in M$.  

Additionally, the predicted probability of rain from the different meteorological stations, $P(y=1 \mid M_i)$, is given by the following list: $[0.1, 0.5, 0.7]$.

A) Define a vector representing the initial prior belief over the models $P_\beta(M_i)$.  
B) Create a function to calculate the marginal model $p_{\text{marginal}}(y)$ for a each $y$.   
C) Make your final decision (or prediction) based on the maximum probability of the Bayesian marginal model.

In [5]:
import numpy as np

# probability of raining for each meteorologist
probability_rain = np.array([0.1, 0.5, 0.7]) # p( y=1 | M )

probability_model = np.array([1-probability_rain, probability_rain]) # p(y|M)

In [6]:
# A) define prior
number_of_meteorologist = 3
prior = np.array([1]*number_of_meteorologist)/number_of_meteorologist  # fill your code
belief = prior

In [7]:
prior

array([0.33333333, 0.33333333, 0.33333333])

In [8]:
def get_marginal_probability(y, belief, p_y_m):
    # fill your code
    marginal_probability = 0
    for i in range(len(belief)):
        marginal_probability += p_y_m[y,i]*belief[i]
    
    return marginal_probability

In [9]:
# B) calculate marginal prediction
p_y0_marginal = get_marginal_probability(0,belief,probability_model) # fill your code # marginal probability of not raining
p_y1_marginal =  get_marginal_probability(1,belief,probability_model)# fill your code # marginal probability of raining
p_y_marginal = [p_y0_marginal, p_y1_marginal] # the whole model

In [10]:
p_y_marginal

[0.5666666666666667, 0.4333333333333333]

In [11]:
# C) calculate the action that the maximum the marginal probability
final_decition = np.argmax(p_y_marginal) # fill your code

In [12]:
print("Our final prediction is :", final_decition)

Our final prediction is : 0


# Step 1.2 Decision According to Utility

In some applications, our predictions (or actions $a$) can significantly effect users.

Consider a scenario where your friends plan a hike based on your weather prediction $a$.
For example if our predictions is rain $a=1$ they dont go for hike.

If we wrongly predict bad weather, the impact of your decision is small, as they simply miss the hike.   
However, incorrectly predicting good weather can have a greater impact, as your friends may end up hiking in unfavorable conditions, which could affect their safety.

One way to adjust our actions according to the effect on the user is to define an additional utility function $U(a, y)$ that outputs a scalar indicating how much our final actions are affected by the outcome $y$.

We can then select the action $a$ that maximizes the expected utility $u(a) = E_{y \sim p_m(y)}[U(a,y)]$ according to our model $p_m$ to estimate the outcome $y$.

The expected utility is defined as follows:
$$ u(a) = E_{y \sim p_m(y)}[U(a,y)] = \sum_y U[a,y] p_m(y) $$

And the final action $a$ is the one that maximizes the expected utility:
$$ a = \arg \max_{a} \;  u(a) = \arg \max_{a} \;  \sum_y U[a,y] p_m(y)  $$



In our example, we have the following utility function $U[a,y]$:

$$
U[a,y] = \begin{bmatrix}
           1  & -10 \\
          -1 & 1
        \end{bmatrix}
$$

So, if our prediction (or action) is correct, i.e., $a = y$ (diagonal of the matrix), then we get a utility of 1.  
If our action is 0 (no rain) and it's actually raining ($y=1$), then we incur a big penalty of -10.  
If our action is 1 (rain) and it's actually not raining ($y=0$), then we incur a small penalty.


**Complete the following steps**:  
A) Fill the function bellow, that calculates the expected utility of an action $a$ according to a utility function U, and a model $p_m$  
B) Produce the final action $a$ that maximise the expected utility according to the marginal model, and the utility function defined above.  
C) Comment on the result. Did you get a different result from before? 

In [13]:
def get_expected_utility(action , model , U):
    """
    Calculate the expected utility of an decision a, according to a model, for specific utility function U
    """
    expected_utility = np.sum([U[action][i]*model[i] for i in range(len(model))]) # fill your code
    
    return expected_utility

In [14]:
U = np.array([[1, -10],
              [-1, 1]])

In [15]:
# find the best 
expected_utility_a_0 = get_expected_utility(0,probability_model,U)# fill your code, calculate expected utility of action a=0 based on the marginal model
expected_utility_a_1 = get_expected_utility(1, probability_model,U)# fill your code, calculate expected utility of action a=1 based on the marginal model
expected_utility = [expected_utility_a_0, expected_utility_a_1] # the whole model

In [16]:
decition_e_utility = np.argmax(expected_utility) # fill your code
print("Our final prediction is :", decition_e_utility)

Our final prediction is : 1


# Step 1.3 Updating Belief

Imagine that after the first day, you observe that is actually rains, i.e, $y_{\text{true}}$=1.  
Then it makes sense to update our belief $P_{\beta}(M)$ about the best model, depending on well the metrologies predicts the weather.

We can update the belief $P_{\beta}(M)$ by calculating the posterior distribution:

$$ p(M_i|y_{\text{true}}) = \frac{p(y_{\text{true}}|M_i) \cdot p(M_i)}{p(y_{\text{true}})} $$

$$ = \frac{p(y_{\text{true}}|M_i) \cdot p(M_i)}{\sum_{j \in M} p(y_{\text{true}}|M_j) \cdot p(M_j)} $$

Then, until we observe some new data, we can use the posterior as our new belief until we observe an new outcome that will change again our belief.

**Complete the following steps**:  
A) Create a function that calculate the posterior probability distribution.  
B) Calculate the posterior is the case that the true outcome is y = 1 (rain).  
C) Compare the values of the posterior with the values of the prior, comment on the results.  
D) Set the belief to be equal to the posterior.  

In [17]:
# A)
def get_posterior(prior, P, y_true):
    """
    Calculate the posterior given a prior belief, a set of predictions, an outcome
    - prior: belief vector so that prior[i] is the probability of model i being correct
    - P: p(y|m) P[y][m] is the probability the m-th model assigns to the y-th outcome
    - outcome: actual outcome
    """
    # fill your code
    posterior = np.zeros(len(prior))
    for i in range(len(prior)):
        for j in range(len(prior)):
            posterior[i] = P[y_true][i]*prior[i]/(P[y_true][i]*prior[i] + P[y_true][j]*prior[j])    
    return posterior

In [18]:
y_true = 1
prior

array([0.33333333, 0.33333333, 0.33333333])

In [19]:
# B) calculate the posterior, based on the true outcome y_true, and the old model
posterior = get_posterior(prior=prior, P=probability_model, y_true=1) # fill your code
posterior

array([0.125     , 0.41666667, 0.5       ])

In [20]:
probability_model

array([[0.9, 0.5, 0.3],
       [0.1, 0.5, 0.7]])

In [21]:
# C) compare prior and posterior
print("Prior distribution:", prior)
print("Posterior distribution:", posterior)

# try to add some comment? what you observe?
## We trust M1 less because it predicted to not rain with 90% and was wrong, thats why the trust went down to 0.125
## Model 3 was the most accurate, so the trust went up to 0.5 from 0.33
## Model 2 said it was 50% chance of rain and it rained, so the trust went up to 0.416 from 0.33

Prior distribution: [0.33333333 0.33333333 0.33333333]
Posterior distribution: [0.125      0.41666667 0.5       ]


In [22]:
# D) update belief
belief = posterior# fill your code

# Step 1.3 Sequential decision-making

To wrap up everything above, consider the case that we sequentially have to produce our actions $a$ based on our belief about the model $P_{\beta}(m)$ and the predictions of the meteorologist. 

Consider for example, 3 consecutive days we sequential get prediction from the different meteorologist.  
At each day we have to produce a predictions (or action $a$).  
After each day we also observe the true outcome, so we update our belief to make the decisions the next day.

**Complete the following steps**:  
Iterate over the different days and:  
A) Calculate the marginal prediction model based on the current belief  
B) Select the action that maximise the marginal prediction model    
C) Select the action that maximise the expected utility based on the marginal prediction model     
D) Calculate posterior based on the true outcome and use it as a belief for the next day.  
F) Comment on the final results

In [23]:
T = 3 # number of time steps
n_models = 3 # number of models

# build predictions for each station of rain probability
predictions = np.array( 
                       [[0.1, 0.4, 0.7], # day 1
                        [0.1, 0.1, 0.8], # day 2
                        [0.3, 0.6, 0.9]] # day 3
                      )


true_y = [0, 0, 1];
n_outcomes = 2 # 0 = no rain, 1 = rain

In [24]:
prior = np.array([1]*n_models)/n_models
belief = prior
for t in range(T):
    # A) get marginal model based on the belief
    probability_rain = predictions[t]
    probability_model = np.array([1-probability_rain, probability_rain])
    p_y0_marginal = get_marginal_probability(0, belief, probability_model)
    p_y_1_marginal = get_marginal_probability(1, belief, probability_model)
    
    p_y_marginal = [p_y0_marginal, p_y_1_marginal]
    
    # B) Select the action that maximizes the marginal prediction model  
    action_marginal = np.argmax(p_y_marginal) 
    
    # C) Select the action that maximizes the expected utility  
    actions_expected_utility = [get_expected_utility(0, p_y_marginal, U), get_expected_utility(1, p_y_marginal, U)]
    final_actions_utility = np.argmax(actions_expected_utility)
    
    # D) Calculate posterior based on the true outcome and use it as a belief for the next day.
    old_prior = belief
    posterior = get_posterior(belief, probability_model, true_y[t])
    belief = posterior
    
    print(f"-------iteration day {t}")
    print("Prior  p(m) = ", old_prior)
    print("Predictions p(y=1|m) = ", predictions[t])
    print("True outcome = ", true_y[t])
    print("Marginal prediction, p_marginal(y=1) = ", p_y_1_marginal)
    print("Action that maximise the marginal model:", action_marginal)
    print("Action that maximise the expected utility U according to the marginal model:", final_actions_utility)
    print("Posterior:", posterior)
    print("\n")

-------iteration day 0
Prior  p(m) =  [0.33333333 0.33333333 0.33333333]
Predictions p(y=1|m) =  [0.1 0.4 0.7]
True outcome =  0
Marginal prediction, p_marginal(y=1) =  0.39999999999999997
Action that maximise the marginal model: 0
Action that maximise the expected utility U according to the marginal model: 1
Posterior: [0.75       0.66666667 0.5       ]


-------iteration day 1
Prior  p(m) =  [0.75       0.66666667 0.5       ]
Predictions p(y=1|m) =  [0.1 0.1 0.8]
True outcome =  0
Marginal prediction, p_marginal(y=1) =  0.5416666666666667
Action that maximise the marginal model: 0
Action that maximise the expected utility U according to the marginal model: 1
Posterior: [0.87096774 0.85714286 0.5       ]


-------iteration day 2
Prior  p(m) =  [0.87096774 0.85714286 0.5       ]
Predictions p(y=1|m) =  [0.3 0.6 0.9]
True outcome =  1
Marginal prediction, p_marginal(y=1) =  1.2255760368663595
Action that maximise the marginal model: 1
Action that maximise the expected utility U accordin

**E) What is the best meteorologist according to the posterior distribution after process?**

The second one is the best one. It leads to the right thing every time. But is not confident with the predictions.

# Step 1.4 Decision Based on Maximum a posteriori (MAP)

Another way to make decisions is to select the model ("meteorologist") that performs the best according to our posterior distribution.

More specifically, in each step, we can choose the model that maximizes the posterior:
      $$m^{\star} = \arg \max_{m} p(m|Data) $$

And then obtain the best action according to that model $p(m|Data)$ instead of the marginal model.
1. Obtain the action with the maximum probability according to the best model $p(y|m^{\star})$.
2. The second option is to select the action that maximizes the expected utility based on the best model $p(y|m^{\star})$.

**Complete the following steps**:   
A) Select the model $m^{\star}$ with the maximum posterior (MAP estimator)   
B) Calculate the action that with the maximum probability according to the model $p(y|m^{\star})$  
C) Calculate the action that with the maximum expected utility, according to the model $p(y|m^{\star} )$  

In [25]:
predictions = [0.3, 0.3, 0.6]

In [37]:
#A) 
map_estimator = np.argmax(posterior) # fill your code

In [38]:
# B)
p_y_1 = predictions[map_estimator] # fill your code
p_y_map = [1-p_y_1, p_y_1]

action_marginal = np.argmax(p_y_map) # fill your code
action_marginal

0

In [39]:
# C) expected utility
u_a_1 = get_expected_utility(1,p_y_map , U) # fill your code
u_a_0 = get_expected_utility(0, p_y_map, U) # fill your code

final_actions_utility = np.argmax([u_a_0, u_a_1]) # fill your code
final_actions_utility

1